In [1]:
!pip install efficientnet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.8 MB/s eta 0:00:00


In [2]:
import cv2
import os
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from efficientnet.keras import EfficientNetB3, EfficientNetB4
from keras.applications import InceptionV3

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [30]:
# Set the data path and categories
data_path = '/kaggle/input/koa-1500/MedicalExpert-I'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories, labels))

img_size = 256
data = []
label = []

# Load the images and labels
for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (img_size, img_size))
            data.append(resized)
            label.append(label_dict[category])
        except Exception as e:
            print('Exception:', e)

data = np.array(data) / 255.0
data_rgb = np.repeat(data[..., np.newaxis], 3, -1)
label = np.array(label)
new_label = np_utils.to_categorical(label)

# Split the data into train, test, and validation sets
x_train, x_test, y_train, y_test = train_test_split(data_rgb, new_label, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125, random_state=42)

# Define the data generators
datagen_train = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

datagen_val = ImageDataGenerator()

train_generator = datagen_train.flow_from_directory(
    '/kaggle/input/koa-1500/MedicalExpert-I/train',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

val_generator = datagen_val.flow_from_directory(
    '/kaggle/input/koa-1500/MedicalExpert-I/validation',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/koa-1500/MedicalExpert-I/train'

In [21]:
# Define the model architecture
model = Sequential()

from keras.layers import Input
from keras.layers import Input, GlobalAveragePooling2D,GlobalMaxPooling2D, Dropout, Dense, Reshape
from keras.models import Model

# Define the input shape
input_shape = (img_size, img_size, 3)

# EfficientNet B3
base_model1 = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
# EfficientNet B5
base_model2 = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)
# InceptionV3
base_model3 = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Create input tensor
input_tensor = Input(shape=input_shape)

# Build the ensemble model
x1 = base_model1(input_tensor)
x1 = GlobalAveragePooling2D()(x1)
x1 = Dropout(0.5)(x1)
x1 = Dense(1024, activation='relu')(x1)

x2 = base_model2(input_tensor)
x2 = GlobalAveragePooling2D()(x2)
x2 = Dropout(0.5)(x2)
x2 = Dense(1024, activation='relu')(x2)

x3 = base_model3(input_tensor)
x3 = GlobalMaxPooling2D()(x3)
x3 = Dense(1024, activation='relu')(x3)
x3 = Reshape((1, 1, 1024))(x3)  # Reshape the tensor to (1, 1, 1024)
x3 = Dropout(0.5)(x3)

# Concatenate the outputs of the three models
x = Dense(len(categories), activation='softmax')(x1 + x2 + x3)

# Create the ensemble model
model = Model(inputs=input_tensor, outputs=x)

# Create the ensemble model
model = Model(inputs=input_tensor, outputs=x)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_46 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnet-b3 (Functional)   (None, 8, 8, 1536)   10783528    ['input_46[0][0]']               
                                                                                                  
 efficientnet-b4 (Functional)   (None, 8, 8, 1792)   17673816    ['input_46[0][0]']               
                                                                                                  
 inception_v3 (Functional)      (None, 6, 6, 2048)   21802784    ['input_46[0][0]']         

In [24]:
# Set up a checkpoint to save the best model
from keras.callbacks import EarlyStopping
checkpoint = ModelCheckpoint('/kaggle/working/model_ensemble.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

In [25]:
# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [26]:
# Train the model
history = model.fit(train_generator,
                    epochs=90,
                    validation_data=val_generator,
                    #callbacks=[checkpoint])
                    callbacks=[checkpoint, early_stopping])

Epoch 1/90


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/opt/conda/lib/pyt

In [ ]:
# Load the best weights
model.load_weights('/kaggle/working/model_ensemble.h5')

In [ ]:
# Plot the training loss and accuracy
N = len(history.history['loss'])
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("EfficientNet_Model")

In [ ]:
# Evaluate the model on the test set
val_loss, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", val_loss)
print("Test accuracy:", val_accuracy)

# Get the predictions for the test set
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=-1)

# Generate the classification report
report = classification_report(np.argmax(y_test, axis=1), predicted_labels)
print(report)

# Plot the confusion matrix
test_labels = np.argmax(y_test, axis=1)
cm = confusion_matrix(test_labels, predicted_labels)
plt.figure()
plot_confusion_matrix(cm, figsize=(12, 8), hide_ticks=True, cmap=plt.cm.Blues)
plt.xticks(range(len(categories)), categories, fontsize=12)
plt.yticks(range(len(categories)), categories, fontsize=12)
plt.show()

# Select a single image from the test set for prediction
X = 32
img_single = x_test[X]
img_single = cv2.resize(img_single, (img_size, img_size))
img_single = np.expand_dims(img_single, 0)

# Make prediction on the single image
predictions_single = model.predict(img_single)
print('A.I predicts:', categories[np.argmax(predictions_single)])
print("Correct prediction for label", np.argmax(y_test[X]), 'is', categories[np.argmax(y_test[X])])

# Display the single image
plt.imshow(np.squeeze(img_single))
plt.grid(False)
plt.show()
